# 一、相关包准备

In [1]:
# 导入包
import jieba
import pandas as pd
import re
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

# 二、加载数据并进行数据预处理

In [2]:
# 加载数据
df = pd.read_excel(r"C:\Users\24756\Desktop\filtered_cnews.train.xlsx")
raw_texts = df['内容'].tolist()
labels = df['类别'].tolist()

# 打印总样本数
print(f"总样本数: {len(raw_texts)}")
# 打印前5个样本验证数据
print("\n原始文本示例：")
for i in range(5):
    print(f"文本{i+1}: {raw_texts[i][:30]}...")  # 只显示前30个字符
# 打印类别及其数量
print("\n类别分布:")
print(pd.Series(labels).value_counts())

总样本数: 30000

原始文本示例：
文本1: 	马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨...
文本2: 	商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，...
文本3: 	冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯1...
文本4: 	辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯...
文本5: 	揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹体坛周报特约记...

类别分布:
房产    5000
体育    5000
财经    5000
科技    5000
家居    5000
教育    5000
dtype: int64


In [3]:
# 加载停用词
stopwords = open(r'C:\Users\24756\stopwords.txt','r',encoding="utf-8").read()
stopwords = stopwords.split('\n')
# 展示前15个停用词
print("停用词: ",",".join(stopwords[:15])) # 第一个是空格
stopwords = set(stopwords) # 转为集合，便于搜索

processed_texts = []
# 文本预处理
for text in raw_texts:
    # 首先，将非中文的内容使用空格代替，并且合并多个空格为一个
    text = re.sub(r"[^\u4e00-\u9fa5]", " ", text)
    text = re.sub(r"\s+", " ", text)
    # jieba分词
    words = jieba.cut(text)
    # 去除停用词
    filtered_words = [word for word in words if word not in stopwords]
    processed_texts.append(filtered_words)
# 打印前2个处理后的文本
print(processed_texts[:2])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\24756\AppData\Local\Temp\jieba.cache


停用词:   ,!,",#,$,%,&,',(,),*,+,,,-,--


Loading model cost 0.503 seconds.
Prefix dict has been built successfully.


[['马晓旭', '意外', '受伤', '国奥', '警惕', '无奈', '大雨', '青睐', '殷家', '军', '记者', '傅亚雨', '沈阳', '报道', '来到', '沈阳', '国奥队', '依然', '摆脱', '雨水', '困扰', '月', '日', '下午', '点', '国奥队', '日常', '训练', '再度', '大雨', '干扰', '无奈', '之下', '队员', '慢跑', '分钟', '草草收场', '日', '上午', '点', '国奥队', '奥体中心', '外场', '训练', '天', '阴沉沉', '气象预报', '显示', '当天', '下午', '沈阳', '大雨', '幸好', '队伍', '上午', '训练', '干扰', '下午', '点', '球队', '抵达', '训练场', '时', '大雨', '几个', '小时', '丝毫', '停下来', '抱', '试一试', '态度', '球队', '当天', '下午', '例行', '训练', '分钟', '天气', '转好', '迹象', '保护', '球员', '国奥队', '中止', '当天', '训练', '全队', '返回', '酒店', '雨', '中', '训练', '足球队', '稀罕', '事', '奥运会', '全队', '变得', '娇贵', '沈阳', '一周', '训练', '国奥队', '保证', '现有', '球员', '出现意外', '伤病', '情况', '影响', '正式', '比赛', '这一', '阶段', '控制', '训练', '受伤', '控制', '感冒', '疾病', '队伍', '放在', '位置', '抵达', '沈阳', '中', '后卫', '冯萧霆', '训练', '冯萧霆', '月', '日', '长春', '患上', '感冒', '参加', '日', '塞尔维亚', '热身赛', '队伍', '介绍', '说', '冯萧霆', '发烧', '症状', '两天', '静养', '休息', '感冒', '恢复', '训练', '冯萧霆', '例子', '国奥队', '对雨中', '训练', '显得', '特别', '谨慎', '担心', '球员', '受凉', '引发', '感冒', '非

# 三、划分数据集

In [4]:
# 类别均衡划分数据集
X_train1, X_temp, y_train1, y_temp = train_test_split(processed_texts, labels, test_size = 0.2, stratify = labels, random_state = 42)
X_val1, X_test1, y_val1, y_test1 = train_test_split(X_temp, y_temp, test_size = 0.5, stratify = y_temp, random_state = 42)

# 打印各数据集大小
print(f"训练集大小: {len(X_train1)}")
print(f"验证集大小: {len(X_val1)}")
print(f"测试集大小: {len(X_test1)}")

# 打印每个类别的分布
print("\n训练集类别分布:\n")
print(pd.Series(y_train1).value_counts())

print("\n验证集类别分布:")
print(pd.Series(y_val1).value_counts())

print("\n测试集类别分布:")
print(pd.Series(y_test1).value_counts())

训练集大小: 24000
验证集大小: 3000
测试集大小: 3000

训练集类别分布:

房产    4000
体育    4000
财经    4000
科技    4000
家居    4000
教育    4000
dtype: int64

验证集类别分布:
教育    500
房产    500
财经    500
体育    500
科技    500
家居    500
dtype: int64

测试集类别分布:
教育    500
房产    500
财经    500
体育    500
科技    500
家居    500
dtype: int64


In [5]:
# 随机划分数据集（类别不均衡）
X_train2, X_temp, y_train2, y_temp = train_test_split(processed_texts, labels, test_size=0.2, random_state=42)
X_val2, X_test2, y_val2, y_test2 = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 打印各数据集大小
print(f"训练集大小: {len(X_train2)}")
print(f"验证集大小: {len(X_val2)}")
print(f"测试集大小: {len(X_test2)}")

# 打印每个类别的分布
print("\n训练集类别分布:")
print(pd.Series(y_train2).value_counts())

print("\n验证集类别分布:")
print(pd.Series(y_val2).value_counts())

print("\n测试集类别分布:")
print(pd.Series(y_test2).value_counts())

训练集大小: 24000
验证集大小: 3000
测试集大小: 3000

训练集类别分布:
教育    4050
财经    4031
体育    4022
家居    3976
科技    3967
房产    3954
dtype: int64

验证集类别分布:
科技    529
房产    513
家居    509
体育    487
教育    486
财经    476
dtype: int64

测试集类别分布:
房产    533
家居    515
科技    504
财经    493
体育    491
教育    464
dtype: int64


# 四、提取特征与特征选择

In [6]:
# 文本向量化
tfidf1 = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)
tfidf2 = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)

# 均衡数据集
X_train_tfidf1 = tfidf1.fit_transform(X_train1)
X_val_tfidf1 = tfidf1.transform(X_val1)
X_test_tfidf1 = tfidf1.transform(X_test1)

# 不均衡数据集
X_train_tfidf2 = tfidf2.fit_transform(X_train2)
X_val_tfidf2 = tfidf2.transform(X_val2)
X_test_tfidf2 = tfidf2.transform(X_test2)

In [7]:
# 利用卡方统计量进行特征选择
# 选择最佳3000个特征
k = 3000

# 对均衡数据集应用卡方检验
selector1 = SelectKBest(chi2, k = k)
X_train_chi1 = selector1.fit_transform(X_train_tfidf1, y_train1)
X_val_chi1 = selector1.transform(X_val_tfidf1)
X_test_chi1 = selector1.transform(X_test_tfidf1)

# 对不均衡数据集应用卡方检验
selector2 = SelectKBest(chi2, k = k)
X_train_chi2 = selector2.fit_transform(X_train_tfidf2, y_train2)
X_val_chi2 = selector2.transform(X_val_tfidf2)
X_test_chi2 = selector2.transform(X_test_tfidf2)

# 五、模型训练与评估

In [8]:
# 定义一个函数评估模型性能
def evaluate_model(y_true, y_pred):
    # 报告混淆矩阵
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    
    # 报告精确度、召回率、F1分数、支持度
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))
    
    # 计算并报告宏平均与微平均F1分数，宏平均更适用于类别均衡，微平均更适用于类别不均衡
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    print(f"Macro F1 Score: {macro_f1:.5f}")
    print(f"Micro F1 Score: {micro_f1:.5f}")

In [9]:
# 不同模型不同数据集进行文本分类，观察分类结果
models = {
    "Naive Bayes": MultinomialNB(),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "GBDT": GradientBoostingClassifier(n_estimators=50, random_state=42)
}

for name, model in models.items():
    # 处理均衡数据集
    model.fit(X_train_chi1, y_train1)
    y_pred_val1 = model.predict(X_val_chi1)
    print(f"\n{name}（类别均衡验证集）:")
    evaluate_model(y_val1, y_pred_val1)
    
    y_pred_test1 = model.predict(X_test_chi1)
    print(f"\n{name}（类别均衡测试集）:")
    evaluate_model(y_test1, y_pred_test1)
    
    # 处理不均衡数据集
    model.fit(X_train_chi2, y_train2)
    y_pred_val2 = model.predict(X_val_chi2)
    print(f"\n{name}（类别不均衡验证集）:")
    evaluate_model(y_val2, y_pred_val2)
   
    y_pred_test2 = model.predict(X_test_chi2)
    print(f"\n{name}（类别不均衡测试集）:")
    evaluate_model(y_test2, y_pred_test2)
    print("\n" + "="*100 + "\n")


Naive Bayes（类别均衡验证集）:
Confusion Matrix:
[[500   0   0   0   0   0]
 [  1 476  11   4   7   1]
 [  0   4 479   3   0  14]
 [  1   7   3 468  18   3]
 [  0   4   1   8 483   4]
 [  0   4  15   9  11 461]]

Classification Report:
              precision    recall  f1-score   support

          体育       1.00      1.00      1.00       500
          家居       0.96      0.95      0.96       500
          房产       0.94      0.96      0.95       500
          教育       0.95      0.94      0.94       500
          科技       0.93      0.97      0.95       500
          财经       0.95      0.92      0.94       500

    accuracy                           0.96      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       0.96      0.96      0.96      3000

Macro F1 Score: 0.95562
Micro F1 Score: 0.95567

Naive Bayes（类别均衡测试集）:
Confusion Matrix:
[[498   0   1   1   0   0]
 [  0 468  22   8   1   1]
 [  0   5 467   8   0  20]
 [  1   6   1 459  28   5]
 [  1   4   5  11 477   2]
 [  0


GBDT（类别不均衡验证集）:
Confusion Matrix:
[[486   0   0   0   1   0]
 [  0 491   0   3  12   3]
 [  0   2 508   0   1   2]
 [  3  12   2 446  13  10]
 [  1   6   2  18 494   8]
 [  0   4  10   7  13 442]]

Classification Report:
              precision    recall  f1-score   support

          体育       0.99      1.00      0.99       487
          家居       0.95      0.96      0.96       509
          房产       0.97      0.99      0.98       513
          教育       0.94      0.92      0.93       486
          科技       0.93      0.93      0.93       529
          财经       0.95      0.93      0.94       476

    accuracy                           0.96      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       0.96      0.96      0.96      3000

Macro F1 Score: 0.95559
Micro F1 Score: 0.95567

GBDT（类别不均衡测试集）:
Confusion Matrix:
[[488   0   0   1   2   0]
 [  1 496   0   2  14   2]
 [  0   0 530   0   1   2]
 [  0  13   0 427  18   6]
 [  1   8   0  23 465   7]
 [  0   6   8   1

In [10]:
# 总得来看，测试集结果与验证集相差不大，分类准确率较好的模型是GBDT